In [1]:
from typing import Tuple
import SourceInv as Si
import Fault as fa
import numpy as np
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import makeFault3D as fd
import math
import TriangularPatches as TriP
import gps
import insar
from point import parallelline as pll

import multifaultsolve as slove
name = 'F000'
mark = 'Maduo'
lono = 98.5    #143  #
lato = 34.6    #38.2 #

Read the fault file, the fault is from two file

In [2]:
st=0.0
et=1.0
workpath='3e19_7e19_K10'

In [ ]:
name = np.loadtxt('./example_test/Maduo/gps_post/20231002/gps_'+str(st)+'_'+str(et)+'_post_disp.txt',comments='#',usecols=(0),unpack=True,dtype=str)
origps = np.loadtxt('./example_test/Maduo/gps_post/20231002/gps_'+str(st)+'_'+str(et)+'_post_disp.txt', comments='#', usecols=(1,2,3,4,5,6,7,8), unpack=True) # neu
pylithgps =  np.loadtxt('./example_test/Maduo/gps_post/20231002/'+workpath+'/pylith_gps_post_'+str(st)+'_'+str(et)+'_total.dat', comments='#', usecols=(3,4,5), unpack=True) #enu
#neu
origps[2] = origps[2] - pylithgps[1]
origps[3] = origps[3] - pylithgps[0]
origps[4] = origps[4] - pylithgps[2]
flld = open('./example_test/Maduo/gps_post/20231002/'+workpath+'/gps_ori_pylith_post_'+str(st)+'_'+str(et)+'.dat','w')
for i in range(len(name)):
    flld.write('{:7s} {:12.5f} {:12.5f} {:12.5f} {:12.5f} {:12.5f} {:10.5f} {:10.5f} {:10.5f}\n'.format(name[i],origps[0][i],origps[1][i],origps[2][i],origps[3][i],origps[4][i],origps[5][i],origps[6][i],origps[7][i]))


In [3]:
dataGPS = gps.gps('B001', utmzone=None, ellps='WGS84',lon0=lono, lat0=lato)
dataGPS.read_from_neu('./example_test/Maduo/gps_post/20231002/'+workpath+'/gps_ori_pylith_post_'+str(st)+'_'+str(et)+'.dat',factor=1.0)

---------------------------------
---------------------------------
Initialize GPS array B001
Read data from file ./example_test/Maduo/gps_post/20231002/3e19_7e19_K10/gps_ori_pylith_post_0.0_1.0.dat into data set B001
load  48  stations


In [4]:
pylith_insar_des_vel=np.loadtxt('./example_test/Maduo/insar/pylith/'+workpath+'/pylith_insar_post_0.0_1.0_Sen1_des_total.dat',usecols=(2),unpack=True)
pylith_insar_asc_vel=np.loadtxt('./example_test/Maduo/insar/pylith/'+workpath+'/pylith_insar_post_0.0_1.0_Sen1_asc_total.dat',usecols=(2),unpack=True)

In [5]:
datainsar_asc = insar.insar('A001',utmzone=None, ellps='WGS84',lon0=lono, lat0=lato)
dsampledatafile='./example_test/Maduo/insar/Asc20220503/dsample_insar_data_jin_'+mark+'_all_RED_asc.txt'
datainsar_asc.read_from_ascii(dsampledatafile)
datainsar_asc.err=datainsar_asc.err+5.0
datainsar_asc.vel = datainsar_asc.vel - pylith_insar_asc_vel

---------------------------------
---------------------------------
Initialize InSAR data set A001
Indar point:  4618


In [6]:
datainsar_des = insar.insar('A002',utmzone=None, ellps='WGS84',lon0=lono, lat0=lato)
dsampledatafile='./example_test/Maduo/insar/Des20220503/dsample_insar_data_jin_'+mark+'_all_RED_des.txt'
datainsar_des.read_from_ascii(dsampledatafile)
datainsar_des.err=datainsar_des.err+5.0
datainsar_des.vel = datainsar_des.vel - pylith_insar_des_vel

---------------------------------
---------------------------------
Initialize InSAR data set A002
Indar point:  3748


In [7]:
#dip = np.array([[8.6,25],[85,7],[30,20],[60,6]])
#dip = np.array([[8.6,25],[65,5],[50,16],[50,6]])
dip = np.array([[6.16,83]]) # 3.1 #3.7 #4.3 #4.93 #5.55 #6.16 #6.8 #7.4 #
Fd1 = fd.Fault_Grid('F001', utmzone=None, ellps='WGS84',lon0=lono, lat0=lato)
Fd1.file2trace('./example_test/Maduo/fault_Maduo_eq_F1.txt')
Fd1.cal_xyi_from_one_more(dip,dwidth = 4 ,dlength = 4,xaxis='x')

Fd1.buildPatch_from_xi()


---------------------------------
Initializing fault F001
Fault strike degree:  [-92.58877551 -92.56214598 -92.50540519 -86.37456814 -76.1073404
 -75.96505773 -75.9852778  -75.93186298 -75.75933109 -66.7409219
 -66.61723478 -84.20690712 -84.09551808 -84.05001545]
Fault length(Km):  [ 6.89169542  9.18936768  9.18971271 13.79971094 23.62651522  4.72477503
  9.44533045 14.16279214 32.35281927  8.1619137   9.74232812  7.06850952
  9.19217311 23.41701748]
Fault total length(Km):  180.964660787948
Fault width(Km):  [50.54593574]
Fault Depth(Km):  [50.169174]
Patches shape: 45 / 12


In [9]:
dip = np.array([[0.88,89]])#3.7 #0.44 #0.53 0.7 0.79 0.88,1.05
Fd2 = fd.Fault_Grid('F002',utmzone=None, ellps='WGS84',lon0=lono, lat0=lato )
Fd2.file2trace('./example_test/Maduo/fault_Maduo_eq_F2.txt')
Fd2.cal_xyi_from_one_more(dip,dwidth = 4 ,dlength = 4,xaxis='x')
Fd2.buildPatch_from_xi()


---------------------------------
Initializing fault F002
Fault strike degree:  [-88.202237   -88.17381779 -88.15960744 -85.98625165 -68.81222127
 -58.47599201]
Fault length(Km):  [6.9217102  2.3071933  2.30717206 9.38448992 3.28080006 7.7657092 ]
Fault total length(Km):  31.96707472433155
Fault width(Km):  [50.42284588]
Fault Depth(Km):  [50.41516624]
Patches shape: 8 / 12


Make Green function , Laplace matrix for each fault. 
Laplace matrix sugguest use the Xu etal.,2015. 


In [10]:
Tf1 = TriP.TriangularPatches('F001', utmzone=None, ellps='WGS84',lon0=lono, lat0=lato)
Tf1.patches2triangles(Fd1, numberOfTriangles=2)
Tf2 = TriP.TriangularPatches('F002', utmzone=None, ellps='WGS84',lon0=lono, lat0=lato)
Tf2.patches2triangles(Fd2, numberOfTriangles=2)

---------------------------------
Initializing fault F001
---------------------------------
Initializing fault F002


In [ ]:
##dataGPS.buildDiagCd()
#Tf.buildGFs(dataGPS,slipdir='sdt')
#faults=[Fd1,Fd2]
datas=[datainsar_asc,datainsar_des,dataGPS]
#datas=[datainsar1,dataGPS]
#datas = [datainsar]
faults=[Tf1,Tf2]
for fault in faults:
    for data in datas:
        fault.buildGFs(data,slipdir='sd')
    #fault.buildLaplacian(method='Xu',locked=True)
    fault.buildLaplacian(method='trao',mapmethod='three')

Scale Xiongwei 2022
Cd=(Cdinsar, Cdgps) ---> W = Cd^-1  ---> W_gps / W_insar = W_ratio


$$C_m(i,j) = (\frac{\sigma \lambda_0}{ \lambda })^2 e^{-\frac{||i,j||_2}{ \lambda }}$$
$$
\textbf{m}_{post} = \textbf{m}_{prior} + (\textbf{G}^t \textbf{C}_d^{-1} \textbf{G})^{-1} \textbf{G}^t \textbf{C}_d^{-1} (\textbf{d} - \textbf{Gm}_{prior})
$$
Builds a model covariance matrix using the equation described in
        Radiguet et al 2010. Here, Sigma and Lambda are lists specifying
        values for the slip directions.
        
$$
C_m(i,j) = (\frac{\sigma \lambda_0}{ \lambda })^2 e^{-\frac{||i,j||_2}{\lambda}}
$$
where,
$$
||i,j||_2
$$ 
is the distance between subfaults,
$$
e^{-\frac{||i,j||_2}{\lambda}}
$$ 
to introduce the correction between nearby parameters,the decreasing exponential stabilizes the solution at large distance, while  allowing important variations in the slip amplitude for nearby subfaults
$$
(\frac{\sigma\lambda_0}{\lambda})^2
$$
weighting factor decease as the lambda increase
lambda increase the more coefficients of the Cm become non-null.

$$\sigma$$ 
  is the a priori standard deviation of model parameters,fixed to 0.5m,
$$\lambda_0$$
is a scaling factor fixed to 10 km (about the size of a subfault
$$\lambda$$  
is the correlation length, fixing the distance over which parameters are correlated.


In [47]:
#w_ratio_list=[1.0,3.0,5.0,8.0,10.0,13.0,15.0,20.0,30.0,50.0,80.0,100.0,150.0,200.0]
w_ratio_list=[15.0]
Maxslip_list = []
cost_list = []
res_list = []
rms_list = []
Variance_list = []
Chi_square_list = []
for w_ratio in w_ratio_list:
    gpsalpha=1.0
    insaralpha=np.sqrt(w_ratio)
    print('Weight ratio gps/insar: ', w_ratio)
    for data in datas:
        if data.dtype in ['insar', 'insartimeseries']:
            data.buildDiagCd(scale=insaralpha)
        elif data.dtype in ['gps', 'opticorr', 'multigps']:
            data.buildCd(direction='enu',scale=gpsalpha)
        else:
            print('Warning!!!')

    #Tf.buildGFs(data,slipdir='sdt')
    for fault in faults:
        fault.assembleGFs(datas,slipdir='sd')
        fault.assembled(datas)
        fault.assembleCd(datas)
        fault.assembleLaplace()
    # construct function
    result = slove.multifaultsolve('Maduo',faults)
    result.assembleGFs()
    result.assembleCm()
    result.assembleLaplace()
    ###solve
    def sm(a):
        sigmaS=1000.0*(10**(a))
        return sigmaS
    Cm_sigmaS = 100.0 
    Cm_lamS = 10.0
    for fault in faults:
        fault.buildCmSlipDirs(Cm_sigmaS, Cm_lamS)
    result = slove.multifaultsolve('Maduo',faults)
    result.assembleGFs()
    result.assembleCm()
    result.GeneralizedLeastSquareSoln(method='jovt')
    result.distributem(verbose=True)
    for data in datas:
        data.buildsynth(faults,direction='sd')
        rms_list.append(data.getRMS())
        Variance_list.append(data.getVariance())
        Chi_square_list.append(data.getChi_square())
    Maxslip = np.max(result.mpost)
    costmf = result.cost
    restmf = result.residual
    cost_list.append(costmf)
    Maxslip_list.append(Maxslip)
    res_list.append(restmf)

Weight ratio gps/insar:  13.0


---------------------------------
---------------------------------
Assembling G for fault F001
Dealing with A001 of type insar
Dealing with A002 of type insar
Dealing with B001 of type gps
---------------------------------
---------------------------------
Assembling d vector
Number of data ： 8510
Dealing with data A001
Dealing with data A002
Dealing with data B001
---------------------------------
---------------------------------
Assembling G for fault F002
Dealing with A001 of type insar
Dealing with A002 of type insar
Dealing with B001 of type gps
---------------------------------
---------------------------------
Assembling d vector
Number of data ： 8510
Dealing with data A001
Dealing with data A002
Dealing with data B001
---------------------------------
---------------------------------
Initializing solver object
Number of data: 8510
Number of parameters: 1452
-----------------
Fault Name                    ||Strike Slip ||Dip Slip    ||Tensile     ||Coupling    ||Extra Parms 


In [3]:
resultpath='pylith/'+workpath + '/two_fault'

In [48]:
rmsarray=np.array(rms_list)[:,1].reshape(len(w_ratio_list),3)
Variancearray=np.array(Variance_list)[:,1].reshape(len(w_ratio_list),3)
Chi_squarearray=np.array(Chi_square_list)[:,1].reshape(len(w_ratio_list),3)

misfitfile = './example_test/Maduo/insar/'+resultpath+'/fix_max_misfit_insar_res5_Cm_'+str(Cm_sigmaS)+'_'+str(Cm_lamS)+'_w_ratio_D25_13.txt'
misfitdata=np.c_[np.array(w_ratio_list),np.array(cost_list),np.array(Maxslip_list),np.array(res_list),rmsarray,Variancearray,Chi_squarearray]
np.savetxt(misfitfile,misfitdata,fmt='%15.4f')

In [ ]:
result.distributem(verbose=True)

分配断层的数据

In [147]:
#two fault 
method='Rai' #L-curve
pathm='TED'
Depth='D50'
Cm_sigmaS = 100.0 
Cm_lamS = 10.0
w_ratio = 15.0
filemark = '_m'+method+'_Cms'+str(Cm_sigmaS)+'_Cml'+str(Cm_lamS)+'_D'+Depth+'_R' + str(w_ratio)
for i in range(len(faults)):
    savPatchfile='./example_test/Maduo/insar/'+resultpath+'/patch3D_'+pathm+'_insar_'+mark+'_strike_F'+ str(i) + filemark +'.txt'
    faults[i].writePatches2File(savPatchfile,add_slip='strikeslip',scale=0.001)
    savPatchfile='./example_test/Maduo/insar/'+resultpath+'/patch3D_'+pathm+'_insar_'+mark+'_dip_F'+ str(i) +filemark+'.txt'
    faults[i].writePatches2File(savPatchfile,add_slip='dipslip',scale=0.001)
    savPatchfile='./example_test/Maduo/insar/'+resultpath+'/patch3D_'+pathm+'_insar_'+mark+'_total_F'+ str(i) + filemark +'.txt'
    faults[i].writePatches2File(savPatchfile,add_slip='total',scale=0.001)
savPatchfile='./example_test/Maduo/insar/'+resultpath+'/patch3D_'+pathm+'_insar_'+mark+'_total_F12'+ filemark +'.txt'
result.writePatches2File(savPatchfile,add_slip='total',scale=0.001)
#result.writeFaultgeometry(path='./example_test/Maduo/',stype='Relax',clon=lono,clat=lato,scale=0.001)

Writing geometry to file ./example_test/Maduo/insar/pylith/3e19_7e19_K10/two_fault/patch3D_TED_insar_Maduo_strike_F0_mRai_Cms100.0_Cml10.0_DD60_R13.0.txt
Writing geometry to file ./example_test/Maduo/insar/pylith/3e19_7e19_K10/two_fault/patch3D_TED_insar_Maduo_dip_F0_mRai_Cms100.0_Cml10.0_DD60_R13.0.txt
Writing geometry to file ./example_test/Maduo/insar/pylith/3e19_7e19_K10/two_fault/patch3D_TED_insar_Maduo_total_F0_mRai_Cms100.0_Cml10.0_DD60_R13.0.txt
Writing geometry to file ./example_test/Maduo/insar/pylith/3e19_7e19_K10/two_fault/patch3D_TED_insar_Maduo_strike_F1_mRai_Cms100.0_Cml10.0_DD60_R13.0.txt
Writing geometry to file ./example_test/Maduo/insar/pylith/3e19_7e19_K10/two_fault/patch3D_TED_insar_Maduo_dip_F1_mRai_Cms100.0_Cml10.0_DD60_R13.0.txt
Writing geometry to file ./example_test/Maduo/insar/pylith/3e19_7e19_K10/two_fault/patch3D_TED_insar_Maduo_total_F1_mRai_Cms100.0_Cml10.0_DD60_R13.0.txt
Writing geometry to file ./example_test/Maduo/insar/pylith/3e19_7e19_K10/two_fault/p

In [148]:
insartype='des'
datainsar_des.buildsynth(faults,direction='sd')
syndatafile='./example_test/Maduo/insar/'+resultpath+'/Insar_org_data_'+insartype + filemark +'.txt'
datainsar_des.write2file(syndatafile,data='data')
syndatafile='./example_test/Maduo/insar/'+resultpath+'/Insar_syn_data_'+insartype + filemark +'.txt'
datainsar_des.write2file(syndatafile,data='synth')
errdatafile='./example_test/Maduo/insar/'+resultpath+'/Insar_err_data_'+insartype + filemark +'.txt'
datainsar_des.write2file(errdatafile,data='resid')

In [149]:
insartype='asc'
datainsar_asc.buildsynth(faults,direction='sd')
syndatafile='./example_test/Maduo/insar/'+resultpath+'/Insar_org_data_'+insartype+ filemark +'.txt'
datainsar_asc.write2file(syndatafile,data='data')
syndatafile='./example_test/Maduo/insar/'+resultpath+'/Insar_syn_data_'+insartype+ filemark +'.txt'
datainsar_asc.write2file(syndatafile,data='synth')
errdatafile='./example_test/Maduo/insar/'+resultpath+'/Insar_err_data_'+insartype+ filemark +'.txt'
datainsar_asc.write2file(errdatafile,data='resid')

In [150]:
dataGPS.buildsynth(faults,direction='sd')
syndatafile='./example_test/Maduo/insar/'+resultpath+'/GPS_org_data'+ filemark +'.txt'
dataGPS.write2file(syndatafile,data='data')
syndatafile='./example_test/Maduo/insar/'+resultpath+'/GPS_syn_data'+ filemark +'.txt'
dataGPS.write2file(syndatafile,data='synth')
resdatafile='./example_test/Maduo/insar/'+resultpath+'/GPS_res_data'+ filemark +'.txt'
dataGPS.write2file(resdatafile,data='res')

Write data set B001 to file .//./example_test/Maduo/insar/pylith/3e19_7e19_K10/two_fault/GPS_org_data_mRai_Cms100.0_Cml10.0_DD60_R13.0.txt
Write synth set B001 to file .//./example_test/Maduo/insar/pylith/3e19_7e19_K10/two_fault/GPS_syn_data_mRai_Cms100.0_Cml10.0_DD60_R13.0.txt
Write res set B001 to file .//./example_test/Maduo/insar/pylith/3e19_7e19_K10/two_fault/GPS_res_data_mRai_Cms100.0_Cml10.0_DD60_R13.0.txt
